# Chapter 12: Model Predictive Control

## Example 12.10: PID_controller_tuned_with_GPC.sce

In [ ]:
// PID controller, tuned with GPC, as discussed in Example 12.5 on page 452.
// 12.10

exec('gpc_pid.sci',-1);
exec('zpowk.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('polsize.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);

A = [1 -1.95 0.935];
B=-0.015;
C=1;
degA=2;
degB=0;
degC=0;
N1=1;
N2=5;
Nu=2;
gamm=0.05;
gamma_y=1;
lambda=0.02;

[Kp,Ki,Kd] = ...
gpc_pid(A,degA,B,degB,C,degC,N1,N2,Nu,lambda,gamm,gamma_y)


## Example 12.11: Predictive_PID_tuned_with_GPC.sci

In [ ]:
// Predictive PID, tuned with GPC, as explained in Sec. 12.2.3.
// 12.11

function [Kp,Ki,Kd] = ...
gpc_pid(A,dA,B,dB,C,dC,N1,N2,Nu,lambda,gamm,gamma_y)
Adelta=convol(A,[1 -1]); G=[];
for i=N1:N2
    zi=zpowk(i);
    [E,dE,F,dF]=xdync(Adelta,dA+1,zi,i,C,dC);
    [Gtilda,dGtilda,Gbar,dGbar] = ...
      xdync(C,dC,zi,i,E*B,dE+dB);
    for j = 1:i, Gtilda1(j)=Gtilda(i+1-j); end
    Gtilda2 = Gtilda1.'; // Added because Scilab forms a column vecor 
    // while Matlab forms a row vector, by default
        if i<=Nu-1
       G=[G;[Gtilda2,zeros(1,Nu-i)]];
    else
       G=[G;Gtilda2(1:Nu)];
    end
end
es=sum(C)/sum(A); gs=sum(B)/sum(A); F_s=es*A; G_s=[];
for i=1:Nu
    if ((Nu - i) == 0)
    row=gs*ones(1,i);
    else
    row=gs*ones(1,i); row=[row,zeros(Nu-i,Nu-i)];
    end;
    G_s=[G_s;row];
end
lambda_mat=lambda*(diag(ones(1,Nu)));
gamma_mat=gamm*(diag(ones(1,Nu)));
gamma_y_mat=gamma_y*(diag(ones(1,N2-N1+1))); 
mat1=inv(G'*gamma_y_mat*G+lambda_mat+G_s'*gamma_mat*G_s);
mat2=mat1*(G'*gamma_y_mat);
mat2_s=mat1*(G_s'*gamma_mat);
h_s=sum(mat2_s(1,:)); h=mat2(1,:);
T=C; R=C*(sum(h(:))+h_s); S=0;
for i=N1:N2
    zi=zpowk(i);
    [E,dE,F,dF]=xdync(Adelta,dA+1,zi,i,C,dC);
    [Gtilda,dGtilda,Gbar,dGbar]=...
      xdync(C,dC,zi,i,E*B,dE+dB);
    S=S+F*h(i);
end
S=S+F_s*h_s;
if length(A)==3
   Kp=S(1)-R-S(3); Ki=R; Kd=S(3);
else
   Kp=S(1)-R; Ki=R; Kd=0;
end

endfunction; 

## Example 12.1: Model_derivation_for_GPC_design.sce

In [ ]:
// Model derivation for GPC design in Example 12.1 on page 439.
// 12.1

exec('xdync.sci',-1);
exec('polmul.sci',-1);
exec('flip.sci',-1);
exec('rowjoin.sci',-1);
exec('polsize.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);

// Camacho and Bordon's GPC example; model formation

A=[1 -0.8]; dA=1; B=[0.4 0.6]; dB=1; N=3; k=1;
D=[1 -1]; dD=1; AD=convol(A,D); dAD=dA+1; Nu=N+k;
zj = 1; dzj = 0; G = zeros(Nu); 
H1 = zeros(Nu,k-1+dB); H2 = zeros(Nu,dA+1);

for j = 1:Nu,
    zj = convol(zj,[0,1]); dzj = dzj + 1;
    [Fj,dFj,Ej,dEj] = xdync(zj,dzj,AD,dAD,1,0);
    [Gj,dGj] = polmul(B,dB,Ej,dEj);
    G(j,1:dGj) = flip(Gj(1:dGj));
    H1(j,1:k-1+dB) = Gj(dGj+1:dGj+k-1+dB);
    H2(j,1:dA+1) = Fj;
end

G,H1,H2

## Example 12.2: Calculates_the_GPC_law.sci

In [ ]:
// Calculates the GPC law given by Eq. 12.19 on page 441.
// 12.2

function [K,KH1,KH2,Tc,dTc,Sc,dSc,R1,dR1] = ...
gpc_bas(A,dA,B,dB,N,k,rho)
D=[1 -1]; dD=1; AD=convol(A,D); dAD=dA+1; Nu=N+1;
zj = 1; dzj = 0; G = zeros(Nu,Nu); 
H1 = zeros(Nu,k-1+dB); H2 = zeros(Nu,dA+1);
for j = 1:Nu,
    zj = convol(zj,[0,1]); dzj = dzj + 1;
    [Fj,dFj,Ej,dEj] = xdync(zj,dzj,AD,dAD,1,0);
    [Gj,dGj] = polmul(B,dB,Ej,dEj);
    G(j,1:dGj) = flip(Gj(1:dGj));
    H1(j,1:k-1+dB) = Gj(dGj+1:dGj+k-1+dB);
    H2(j,1:dA+1) = Fj;
end
K = inv(G'*G+rho*eye(Nu,Nu))*G';
// Note: inverse need not be calculated
KH1 = K * H1; KH2 = K * H2;
R1 = [1 KH1(1,:)]; dR1 = length(R1)-1;
Sc = KH2(1,:); dSc = length(Sc)-1;
Tc = K(1,:); dTc = length(Tc)-1;
endfunction;

## Example 12.3: GPC_design_for_the_problem_discussed_on_page_441.sce

In [ ]:
// GPC design for the problem discussed in Example 12.2 on page 441.
// 12.3

exec('gpc_bas.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('polsize.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);
exec('polmul.sci',-1);
exec('flip.sci',-1);
exec('filtval.sci',-1);

// Camacho and Bordon's GPC example; Control law
A=[1 -0.8]; dA=1; B=[0.4 0.6]; dB=1; N=3; k=1; rho=0.8;
[K,KH1,KH2,Tc,dTc,Sc,dSc,R1,dR1] = ...
gpc_bas(A,dA,B,dB,N,k,rho)
// C=1; dC=0; [K,KH1,KH2,Tc,dTc,Sc,dSc,R1,dR1] = ...
// gpc_col(A,dA,B,dB,C,dC,N,k,rho)











## Example 12.4: GPC_desig.sce

In [ ]:
// GPC design for the problem discussed in Example 12.3.
// 12.4

exec('gpc_N.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('polsize.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);
exec('polmul.sci',-1);
exec('flip.sci',-1);

A=[1 -0.8]; dA=1; B=[0.4 0.6]; dB=1;
rho = 0.8; k = 1;
N1 = 0; N2 = 3; Nu = 2;

[K,KH1,KH2,Tc,dTc,Sc,dSc,R1,dR1] = ...
gpc_N(A,dA,B,dB,k,N1,N2,Nu,rho)


## Example 12.5: Calculates_the_GPC_law.sci

In [ ]:
// Calculates the GPC law given by Eq. 12.36 on page 446. 
// 12.5

function [K,KH1,KH2,Tc,dTc,Sc,dSc,R1,dR1] = ...
gpc_N(A,dA,B,dB,k,N1,N2,Nu,rho)
D=[1 -1]; dD=1; AD=convol(A,D); dAD=dA+1; 
zj = 1; dzj = 0;
for i = 1:N1+k-1
    zj = convol(zj,[0,1]); dzj = dzj + 1;
end
G = zeros(N2-N1+1,Nu+1); 
H1 = zeros(N2-N1+1,k-1+dB); H2 = zeros(N2-N1+1,dA+1);
for j = k+N1:k+N2
    zj = convol(zj,[0,1]); dzj = dzj + 1;
    [Fj,dFj,Ej,dEj] = xdync(zj,dzj,AD,dAD,1,0);
    [Gj,dGj] = polmul(B,dB,Ej,dEj);
    if (j-k >= Nu)
    G(j-(k+N1-1),1:Nu+1) = flip(Gj(j-k-Nu+1:j-k+1));
else
    G(j-(k+N1-1),1:j-k+1) = flip(Gj(1:j-k+1));
end
    H1(j-(k+N1-1),1:k-1+dB) = Gj(j-k+2:j+dB);
    H2(j-(k+N1-1),1:dA+1) = Fj;
end
K = inv(G'*G+rho*eye(Nu+1,Nu+1))*G';
// Note: inverse need not be calculated
KH1 = K * H1; KH2 = K * H2;
R1 = [1 KH1(1,:)]; dR1 = length(R1)-1;
Sc = KH2(1,:); dSc = length(Sc)-1;
Tc = K(1,:); dTc = length(Tc)-1;
endfunction;

## Example 12.6: Calculates_the_GPC_law.sci

In [ ]:
// Calculates the GPC law given by Eq. 12.36 on page 446.
// 12.6

function [K,KH1,KH2,Tc,dTc,Sc,dSc,R1,dR1] = ...
gpc_col(A,dA,B,dB,C,dC,N,k,rho)
D=[1 -1]; dD = 0; AD=convol(A,D); dAD=dA+1; zj=1; dzj=0; 
Nu = N+1; G=zeros(Nu,Nu); H1=zeros(Nu,2*k+N-2+dB); 
H2 = zeros(Nu,k+N+dA);
for j = 1:Nu,
    zj = convol(zj,[0,1]); dzj = dzj + 1;
    [Fj,dFj,Ej,dEj] = ...
        xdync(zj,dzj,AD,dAD,C,dC);
    [Nj,dNj,Mj,dMj] = ...
        xdync(zj,dzj,C,dC,1,0);
    [Gj,dGj] = polmul(Mj,dMj,Ej,dEj);
    [Gj,dGj] = polmul(Gj,dGj,B,dB);
    [Pj,dPj] = polmul(Mj,dMj,Fj,dFj);
    [Pj,dPj] = poladd(Nj,dNj,Pj,dPj);
    j,Fj,Ej,Mj,Nj,Gj,Pj
    G(j,1:j) = flip(Gj(1:j));
    H1(j,1:dGj-j+1) = Gj(j+1:dGj+1); 
    H2(j,1:dPj+1) = Pj;
end
K = inv(G'*G+rho*eye(Nu,Nu))*G'
// Note: inverse need not be calculated
KH1 = K * H1; KH2 = K * H2;
R1 = [1 KH1(1,:)]; dR1 = length(R1)-1;
Sc = KH2(1,:); dSc = length(Sc)-1;
Tc = K(1,:); dTc = length(Tc)-1;
endfunction;

## Example 12.7: GPC_design_for_viscosity_control.sce

In [ ]:
// GPC design for viscosity control in Example 12.4 on page 446.
// 12.7

exec('gpc_col.sci',-1);
exec('poladd.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('polsize.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);
exec('polmul.sci',-1);
exec('flip.sci',-1);

// GPC control of viscosity problem
A=[1 -0.44]; dA=1; B=[0.51 1.21]; dB=1; N=2; k=1;
C = [1 -0.44]; dC = 1; rho = 1;

[K,KH1,KH2,Tc,dTc,Sc,dSc,R1,dR1] = ...
gpc_col(A,dA,B,dB,C,dC,N,k,rho)

## Example 12.8: GPC_desig.sce

In [ ]:
// GPC design for the problem discussed in Example 12.3.
// 12.8

exec('gpc_Nc.sci',-1);
exec('xdync.sci',-1);
exec('rowjoin.sci',-1);
exec('polsize.sci',-1);
exec('left_prm.sci',-1);
exec('t1calc.sci',-1);
exec('indep.sci',-1);
exec('seshft.sci',-1);
exec('makezero.sci',-1);
exec('move_sci.sci',-1);
exec('colsplit.sci',-1);
exec('clcoef.sci',-1);
exec('cindep.sci',-1);
exec('polmul.sci',-1);
exec('poladd.sci',-1);
exec('flip.sci',-1);

A=[1 -0.44]; dA=1; B=[0.51 1.21]; dB=1;
C = [1 -0.44]; dC = 1;
k=1; N1 = 0; N2 = 2; Nu = 0; rho = 1;

[K,KH1,KH2,Tc,dTc,Sc,dSc,R1,dR1] = ...
gpc_Nc(A,dA,B,dB,C,dC,k,N1,N2,Nu,rho)


## Example 12.9: Calculates_the_GPC_law.sci

In [ ]:
// Calculates the GPC law for different prediction and control horizons
// 12.9

function [K,KH1,KH2,Tc,dTc,Sc,dSc,R1,dR1] = ...
gpc_Nc(A,dA,B,dB,C,dC,k,N1,N2,Nu,rho)
D=[1 -1]; dD=1; AD=convol(A,D); dAD=dA+1; 
zj = 1; dzj = 0;
for i = 1:N1+k-1
    zj = convol(zj,[0,1]); dzj = dzj + 1;
end
M = 2*k+N2-2+dB;  P = max(k+N2+dA-1,dC-1)
G = zeros(N2-N1+1,Nu+1); H1 = zeros(N2-N1+1,M); 
H2 = zeros(N2-N1+1,P+1);
for j = k+N1:k+N2
    zj = convol(zj,[0,1]); dzj = dzj + 1;
    [Fj,dFj,Ej,dEj] = xdync(zj,dzj,AD,dAD,C,dC);
    [Nj,dNj,Mj,dMj] = xdync(zj,dzj,C,dC,1,0);
    [Gj,dGj] = polmul(Mj,dMj,Ej,dEj);
    [Gj,dGj] = polmul(Gj,dGj,B,dB);
    [Pj,dPj] = polmul(Mj,dMj,Fj,dFj);
    [Pj,dPj] = poladd(Nj,dNj,Pj,dPj);
    if (j-k >= Nu)
    G(j-(k+N1-1),1:Nu+1) = flip(Gj(j-k-Nu+1:j-k+1));
else
    G(j-(k+N1-1),1:j-k+1) = flip(Gj(1:j-k+1));
end
    H1(j-(k+N1-1),1:j+k-2+dB) = Gj(j-k+2:2*j+dB-1);
    dPj = max(j-1+dA,dC-1);
    H2(j-(k+N1-1),1:dPj+1) = Pj;
end
K = inv(G'*G+rho*eye(Nu+1,Nu+1))*G';
// Note: inverse need not be calculated
KH1 = K * H1; KH2 = K * H2;
R1 = [1 KH1(1,:)]; dR1 = length(R1)-1;
Sc = KH2(1,:); dSc = length(Sc)-1;
Tc = K(1,:); dTc = length(Tc)-1;
endfunction;